In [6]:
import pandas as pd
import dotenv
from dotenv import load_dotenv
import openai
import os
import json
load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

## Criando um Assistant ##

In [8]:
assistant = openai.beta.assistants.create(
    name='Profissional de dados',
    instructions='Fornecer e analiser dados e gerar codigos em python e analisar estatisticas',
    tools=[{'type':'code_interpreter'}],
    model='gpt-3.5-turbo-0125'
  
 )

## Criando um Thread ##

In [11]:
thread = openai.beta.threads.create()

message = openai.beta.threads.messages.create(
    thread_id = thread.id,
    role = 'user',
    content='se eu jogar um dado honesto 1000 vezes qual a probabliade de numeros que vou obter 150 vezes o numero 6'

)

## Rodando a thread no assistant ##

In [12]:
run = openai.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions='O nome do usuario é Adriano Soares e ele e um usuario Premium'
)

## Aguardar a thread rodar ##

In [13]:
import time

while run.status in ['queued', 'in_progress','cancelling']:
    time.sleep(1)
    run = openai.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )


In [14]:
run.status


'completed'

## Verificar Resposta ##

In [16]:
if run.status == 'completed':
    mensagens = openai.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(mensagens)

else:
    print('errro', run.status)

SyncCursorPage[Message](data=[Message(id='msg_BkEEnbW9LlT1BvD0Q2r1sJm0', assistant_id='asst_7JGLej03JaZWfeI5OO6T5Ljc', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='A probabilidade de obter o número 6 exatamente 150 vezes ao lançar um dado honesto 1000 vezes é aproximadamente 1.26%.'), type='text')], created_at=1737468340, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_uTta2HZbFMGVHWAnh7BXSewO', status=None, thread_id='thread_9VF61WHAAA8WLvEGBjf0ACxq'), Message(id='msg_2xsszIKhnH9QZBwEbd5uvvH1', assistant_id='asst_7JGLej03JaZWfeI5OO6T5Ljc', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Para calcular a probabilidade de obter o número 6 exatamente 150 vezes ao lançar um dado honesto 1000 vezes, podemos usar a distribuição binomial. A distribuição binomial é usada para calcular a probabilidade de um evento ocorrer um certo número

In [17]:
print(mensagens.data[0].content[0].text.value)

A probabilidade de obter o número 6 exatamente 150 vezes ao lançar um dado honesto 1000 vezes é aproximadamente 1.26%.


## Analisando Passo a passo ##

In [18]:
run_steps = openai.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

In [19]:
for step in run_steps.data:
    print('==== step', step.step_details.type)

==== step message_creation
==== step tool_calls
==== step message_creation


In [29]:
for step in run_steps.data:
    print('=== Step:', step.step_details.type)
    if step.step_details.type == 'tool_calls':
        for tool_call in step.step_details.tool_calls:
            print('-----')
            print(tool_call.code_interpreter.input)
            print('-----')
            print('Result')
            if tool_call.code_interpreter.outputs:
                print(tool_call.code_interpreter.outputs[0].logs)
            else:
                print('No output logs available')
    

=== Step: message_creation
=== Step: tool_calls
-----
from math import comb

n = 1000
k = 150
p = 1/6

probabilidade = comb(n, k) * (p**k) * ((1-p)**(n-k))
probabilidade
-----
Result
No output logs available
=== Step: message_creation


In [ ]:
for step in run_steps.data[::-1]:
    print('\n=== Step:', step.step_details.type)
    if step.step_details.type == 'tool_calls':
        for tool_call in step.step_details.tool_calls:
            if tool_call.type == 'file_search':
                print(tool_call)
            else:
                print('-----')
                print(tool_call.code_interpreter.input)
                print('-----')
                print('Result')
                print(tool_call.code_interpreter.outputs[0].logs)
    if step.step_details.type == 'message_creation':
        message = openai.beta.threads.messages.retrieve(
            thread_id=thread.id,
            message_id=step.step_details.message_creation.message_id
        )
        print(message.content[0].text.value)